# Find Location And Fix Addresses Record

## Run this cell to import all the necessary dependencies

In [1]:
import pandas as pd
import numpy as np
import json
# !pip install geopy 
from geopy.geocoders import Nominatim
from geopy.geocoders import GoogleV3

#!pip install -U googlemaps
import googlemaps
from os.path import exists

result_directory = 'results/'

# Part 1

## Run this cell if there is no CSV generated by this script. Otherwise you can skip to part 2

In [2]:
df = pd.read_csv('property_price_register_dublin_2019-2021.csv')
df

,CODE,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC
0,1,01/01/2019,"2 LEIX RD, CABRA, DUBLIN 7",Dublin 7,Dublin,30000.0,1,0,Second-Hand Dwelling house /Apartment
1,2,01/01/2019,"2 WOODALE VIEW, BALLYCULLEN, FIRHOUSE DUBLIN 24",Dublin 24,Dublin,30000.0,1,0,Second-Hand Dwelling house /Apartment
2,3,01/01/2019,"APT 50, EARLSFIELD COURT, FRANCIS ST DUBLIN 8",Dublin 8,Dublin,15000.0,1,0,Second-Hand Dwelling house /Apartment
3,4,02/01/2019,"31 ALL SAINTS PARK, RAHENY, DUBLIN 5",Dublin 5,Dublin,460000.0,0,0,Second-Hand Dwelling house /Apartment
4,5,02/01/2019,"40 THE COURTYARD, CLONSILLA, DUBLIN 15",Dublin 15,Dublin,221500.0,0,0,Second-Hand Dwelling house /Apartment
...,...,...,...,...,...,...,...,...,...
25197,25198,27/05/2021,"APT 307, BLOCK A, HAMPTON LODGE GRACEPARK RD",Dublin 9,Dublin,320000.0,0,0,Second-Hand Dwelling house /Apartment
25198,25199,27/05/2021,"INNSBRUCK, 170A BOTANIC RD, GLASNEVIN DUBLIN 9",Dublin 9,Dublin,585000.0,0,0,Second-Hand Dwelling house /Apartment
25199,25200,28/05/2021,"116A CHERRYWOOD DR, CLONDALKIN, DUBLIN 22",Dublin 22,Dublin,295000.0,0,0,Second-Hand Dwelling house /Apartment
25200,25201,28/05/2021,"189 CROSBIES YARD, OSSORY RD, DUBLIN 1",Dublin 1,Dublin,306000.0,0,0,Second-Hand Dwelling house /Apartment


In [163]:
# get df coulumns
df.columns

Index(['CODE', 'SALE_DATE', 'ADDRESS', 'POSTAL_CODE', 'COUNTY', 'SALE_PRICE',
       'IF_MARKET_PRICE', 'IF_VAT_EXCLUDED', 'PROPERTY_DESC'],
      dtype='object')

In [3]:
def is_county_in_address(df):
    counter = 0
    for i in range(len(df)):
        if df.iloc[i]['COUNTY'].lower() not in df.iloc[i]['ADDRESS'].lower():
            counter += 1
    print('Number of addresses that doesn\'t contain county name: ', counter)
    print('Number of addresses that contain county name: ', len(df) - counter)

In [4]:
is_county_in_address(df)

Number of addresses that doesn't contain county name:  5141
Number of addresses that contain county name:  20061


Turn off the cache option

In [5]:
df.is_copy = False

In [6]:
# traverse every row in df and check if the address contains county name
# if it does not contain county name, update the address that doesn't contain county name with the address + comma + county name
# don't let copy, just update the original df
for i in range(len(df)):
    if df.iloc[i]['COUNTY'].lower() not in df.iloc[i]['ADDRESS'].lower():
        df.at[i, 'ADDRESS'] = df.iloc[i]['ADDRESS'] + ', ' + df.iloc[i]['COUNTY']

In [7]:
is_county_in_address(df)

Number of addresses that doesn't contain county name:  0
Number of addresses that contain county name:  25202


Turn on the cache option

In [146]:
df.is_copy = True

---

# Part 2

## Load methods to save files in formats json and csv

In [111]:
import json

def save_to_json(data, filename):
    with open(filename, 'w') as f:
        json.dump(data, f, indent=4)

def save_to_csv(df, filename):
    df.to_csv(filename, encoding='utf-8', index = False)

### Instantiating Geopy with Nominatim

In [112]:
nominatim = Nominatim(user_agent="[your project name]")

### Instantiating Geocoding API

In [113]:
gmaps = googlemaps.Client(key='[API_KEY]')

### Instantiating Geopy with GoogleV3

In [179]:
googlev3 = GoogleV3(api_key='[API_KEY]')

### Run this cell to load method find_addresses

In [160]:
# import package datetime
import datetime
import json

def find_addresses(geolocator, df, address='ADDRESS', index='CODE', start=0, end=0, report_error=[], timemout=5):

    dfsize = len(df)
    if end > dfsize or end <= 0:
        end = dfsize
    if start > dfsize:
        start = dfsize
        
    next = start
    
    if address not in df.columns:
        raise Exception('Address column not found in df')
    if index not in df.columns:
        raise Exception('Index column not found in df')

    if not {'LONGITUDE', 'LATITUDE'}.issubset(df.columns):
        df = df.assign(LONGITUDE=0.0, LATITUDE=0.0)
    try:
        while next < end:
            if df.loc[next]['LONGITUDE'] == 0 and df.loc[next]['LATITUDE'] == 0:
                location = geolocator.geocode(df.loc[next][address], timeout=timemout)
                if location:
                    df.at[next, address] = location.address
                    df.at[next, 'LONGITUDE'] = location.longitude
                    df.at[next, 'LATITUDE'] = location.latitude
                else:
                    now = datetime.datetime.now().strftime("%d/%m/%Y %H:%M:%S")
                    report_error.append({'record': df.loc[next, [address, index]].to_json(), 'time': now})
            next += 1
    except Exception as e:
        print(e)
        print('Error at record: ', df.loc[next, [address, index]])
    
    report = {'error': report_error, 'start': start, 'end': end, 'next': next}
    return df, report
    

### Run this cell to load method bot_find_addresses

In [161]:
def bot_find_addresses(geolocator, df, address='ADDRESS', index='CODE', start=0, end=0, batch=1, report_error=[], timemout=5):
    
    df_tmp = df.copy()
    report = []
    diff = end - start
    
    for r in range(batch):
        df_tmp, report = find_addresses(geolocator, df_tmp, 
            address=address, index=index, start=start, end=end, report_error=report_error, timemout=timemout)
        report_error = report['error']
        start = end
        end = end + diff
        filename_csv = '{}properties_info_{}_{}.csv'.format(result_directory, report['start'], report['end'])
        filename_json = '{}report_info_{}_{}.json'.format(result_directory, report['start'], report['end'])
        save_to_json(report, filename_json)
        save_to_csv(df_tmp, filename_csv)
    return df_tmp


#### Run this cell if no CSV has been generated yet

In [201]:

botdf = df.copy()

If there are CSV and JSON files, change the variables start and end manually. The file name contains the start and end values.

In [192]:
#2200 - 1700
start = 20720
end = 22720
report_error=[]

#### If you have generated some CSV files or it is the first time run the cell below

In [191]:

filename_csv = '{}properties_info_{}_{}.csv'.format(result_directory, start, end)
filename_json = '{}report_info_{}_{}.json'.format(result_directory, start, end)

if exists(filename_csv) and exists(filename_json):
    botdf = pd.read_csv(filename_csv)

    with open(filename_json) as f:
        botjson = json.load(f)

    start = botjson['start']
    end = botjson['end']
    report_error = botjson['error']
    print(botdf.loc[:,['CODE','ADDRESS', 'LONGITUDE', 'LATITUDE']].head())
    print(botdf.loc[:,['CODE','ADDRESS', 'LONGITUDE', 'LATITUDE']].tail())
    print('start:', start)
    print('end:', end)
else:
    print('Files {} and {} not found'.format(filename_csv, filename_json))

   CODE                                            ADDRESS  LONGITUDE  \
0     1  2, Leix Road, Cabra East, Cabra East B ED, Dub...  -6.283672   
1     2  24 Wood Dale View, Ballycullen, Baile Átha Cli...  -6.334965   
2     3  Earlsfield Court, Francis St, The Liberties, D...  -6.274056   
3     4  All Saints Park, Raheny, Clontarf East A ED, D...  -6.178369   
4     5  The Courtyard, Blanchardstown-Delwood ED, Clon...  -6.403226   

    LATITUDE  
0  53.361690  
1  53.275939  
2  53.339771  
3  53.377484  
4  53.381423  
        CODE                                            ADDRESS  LONGITUDE  \
25197  25198  APT 307, BLOCK A, HAMPTON LODGE GRACEPARK RD, ...   0.000000   
25198  25199     INNSBRUCK, 170A BOTANIC RD, GLASNEVIN DUBLIN 9   0.000000   
25199  25200  Cherrywood Drive, Clondalkin-Village ED, Clond...  -6.409972   
25200  25201             189 CROSBIES YARD, OSSORY RD, DUBLIN 1   0.000000   
25201  25202  Corduff Gardens, Blanchardstown-Corduff ED, Bl...  -6.372640   

  

#### Run this cell to start searching for addresses.

In [193]:
test_df = bot_find_addresses(googlev3, botdf, start=start, end=end, batch=3, report_error=report_error)
test_df.head()

,CODE,SALE_DATE,ADDRESS,POSTAL_CODE,COUNTY,SALE_PRICE,IF_MARKET_PRICE,IF_VAT_EXCLUDED,PROPERTY_DESC,LONGITUDE,LATITUDE
0,1,01/01/2019,"2, Leix Road, Cabra East, Cabra East B ED, Dub...",Dublin 7,Dublin,30000.0,1,0,Second-Hand Dwelling house /Apartment,-6.283672,53.361690
1,2,01/01/2019,"24 Wood Dale View, Ballycullen, Baile Átha Cli...",Dublin 24,Dublin,30000.0,1,0,Second-Hand Dwelling house /Apartment,-6.334965,53.275939
2,3,01/01/2019,"Earlsfield Court, Francis St, The Liberties, D...",Dublin 8,Dublin,15000.0,1,0,Second-Hand Dwelling house /Apartment,-6.274056,53.339771
3,4,02/01/2019,"All Saints Park, Raheny, Clontarf East A ED, D...",Dublin 5,Dublin,460000.0,0,0,Second-Hand Dwelling house /Apartment,-6.178369,53.377484
4,5,02/01/2019,"The Courtyard, Blanchardstown-Delwood ED, Clon...",Dublin 15,Dublin,221500.0,0,0,Second-Hand Dwelling house /Apartment,-6.403226,53.381423
